In [ ]:
!pip install evaluate nltk

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
import evaluate
import nltk
nltk.download('punkt')

## Load the Models

In [ ]:


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load model from saved folder
model_path = "/content/drive/MyDrive/your_folder_name"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")


## Code to Compute Exact Match, BLEU, and ROUGE

In [ ]:
# Initialize metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
exact_match_count = 0

predictions = []
references = []

# Evaluation loop
for example in test_dataset[:50]:  # or however many examples you want
    inputs = processor(
        images=Image.open(example["image"]).convert("RGB"),
        text=example["instruction"],
        return_tensors="pt"
    ).to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=50)
    pred = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    ref = example["output"]

    predictions.append(pred)
    references.append(ref)

    if pred.strip().lower() == ref.strip().lower():
        exact_match_count += 1

# Compute metrics
bleu_score = bleu.compute(predictions=predictions, references=[[r] for r in references])
rouge_score = rouge.compute(predictions=predictions, references=references)
exact_match = exact_match_count / len(predictions)

print("Exact Match:", exact_match)
print("BLEU:", bleu_score["bleu"])
print("ROUGE-L:", rouge_score["rougeL"])
